In [1]:
from src.data_reader import DataReader
from src.constant import *
import matplotlib.pyplot as plt
import numpy as np
from src.datasets.dataset import Dataset
from src.models.itemknn.itemknn import ItemKNN
import pandas as pd
from src.evaluation import compute_mrr

In [2]:
dataset = Dataset()

In [3]:
item_features = dataset.get_item_features()

In [4]:
item_features

,item_id,feature_category_id,feature_value_id
0,0,56,365
1,0,62,801
2,0,68,351
3,0,33,802
4,0,72,75
...,...,...,...
471746,23690,68,351
471747,23690,55,390
471748,23690,11,109
471749,23690,73,91


In [5]:
oh_cat = pd.get_dummies(item_features[F_CAT], prefix = "cat")

In [6]:
oh_val = pd.get_dummies(item_features[F_VAL], prefix="val")

In [7]:
item_features_oh = item_features.join(oh_cat).join(oh_val)

In [8]:
item_features_oh = item_features_oh.groupby(ITEM_ID).sum()

In [9]:
item_features_oh = item_features_oh.drop([F_VAL, F_CAT], axis=1).reset_index()

In [10]:
item_features_oh

,item_id,cat_1,cat_2,cat_3,cat_4,cat_5,cat_6,cat_7,cat_8,cat_9,...,val_896,val_897,val_898,val_899,val_900,val_901,val_902,val_903,val_904,val_905
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,1,1,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
2,2,0,0,1,1,1,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,3,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
4,4,0,0,1,1,1,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23686,23686,0,0,1,1,1,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
23687,23687,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23688,23688,1,0,1,1,1,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
23689,23689,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
